## Verilerin Sınıflandırma İle Tahmin Etmesi

In [67]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # Veri setini eğitim ve test olarak ayırmak için
from sklearn.pipeline import Pipeline # İş akışlarını yönetmek için
from sklearn.preprocessing import StandardScaler, OneHotEncoder # Veriyi ölçeklendirmek ve kategorik verileri dönüştürmek için
from sklearn.compose import ColumnTransformer # Farklı sütunlara farklı ön işleme adımları uygulamak için
from sklearn.ensemble import RandomForestClassifier # Sınıflandırma modeli olarak rastgele orman
from sklearn.metrics import classification_report, confusion_matrix # Sınıflandırma performansını değerlendirmek için

In [68]:
df = pd.read_csv('hepsiemlak_cleaned_final.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7652 entries, 0 to 7651
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   city          7652 non-null   object
 1   district      7652 non-null   object
 2   neighborhood  7652 non-null   object
 3   room          7652 non-null   int64 
 4   livingroom    7652 non-null   int64 
 5   area          7652 non-null   int64 
 6   age           7652 non-null   int64 
 7   floor         7652 non-null   int64 
 8   price         7652 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 538.2+ KB
None


In [69]:
# Nümerik değişkenlerin minimum, maximum ve çeyreklik değerlerinin bulunması
df['city'] = df['city'].astype('category') # city sütununu kategorik veri tipine dönüştür
df['district'] = df['district'].astype('category') # district sütununu kategorik veri tipine dönüştür
df['neighborhood'] = df['neighborhood'].astype('category') # neighborhood sütununu kategorik veri tipine dönüştür
df['room'] = df['room'].astype('int') # room sütununu tamsayı veri tipine dönüştür
df['livingroom'] = df['livingroom'].astype('int') # livingroom sütununu tamsayı veri tipine dönüştür
df['area'] = df['area'].astype('int') # area sütununu tamsayı veri tipine dönüştür
df['age'] = df['age'].astype('int') # age sütununu tamsayı veri tipine dönüştür
df['floor'] = df['floor'].astype('int') # floor sütununu tamsayı veri tipine dönüştür
df['price'] = df['price'].astype('int') # price sütununu tamsayı veri tipine dönüştür

In [70]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7652 entries, 0 to 7651
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   city          7652 non-null   category
 1   district      7652 non-null   category
 2   neighborhood  7652 non-null   category
 3   room          7652 non-null   int32   
 4   livingroom    7652 non-null   int32   
 5   area          7652 non-null   int32   
 6   age           7652 non-null   int32   
 7   floor         7652 non-null   int32   
 8   price         7652 non-null   int32   
dtypes: category(3), int32(6)
memory usage: 307.5 KB
None


In [71]:
# Veri ön işleme adımlarını tanımlama
categorical_features = ['city', 'district', 'neighborhood'] # Kategorik değişkenler
numerical_features = ['room', 'livingroom', 'area', 'age', 'floor'] # Nümerik değişkenler

In [72]:
# Veri ön işleme adımlarını tanımlama
full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), numerical_features), # Nümerik değişkenler için ölçeklendirme
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # Kategorik değişkenler için one-hot encoding
])

In [73]:
# Veri setini eğitim ve test olarak ayırma
X = df.drop('price', axis=1) # Özellikler (bağımsız değişkenler)
y = df['price'] # Hedef değişken (bağımlı değişken)

In [74]:
bins = [x for x in range(0, 90000, 10000) ]
labels = [x for x in range(1,9)]
print(bins)
print(labels)

[0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000]
[1, 2, 3, 4, 5, 6, 7, 8]


In [75]:
y = pd.cut(y, bins=bins, labels=labels) # Hedef değişkeni kategorik hale getirme


In [76]:
print(y.unique())  # Kategorik hedef değişkenin benzersiz etiketlerini yazdırma

[2, 3, 4, 5, 1, 6, 7]
Categories (8, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7 < 8]


In [77]:
# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Veri setini eğitim ve test olarak ayırma # random_state=42 sabit bir bölme sağlar

In [78]:
# Model tanımlaması
model = Pipeline([
    ('preparation', full_pipeline), # Ön işleme adımları
    ('model', RandomForestClassifier(n_estimators=100)) # Model tanımlaması
])

In [79]:
model.fit(X_train, y_train) # Modeli eğitim verisi ile eğitme

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['room', 'livingroom', 'area',
                                                   'age', 'floor']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city', 'district',
                                                   'neighborhood'])])),
                ('model', RandomForestClassifier())])

In [80]:
y_pred = model.predict(X_test) # Test verisi üzerinde tahmin yapma

In [81]:
print(confusion_matrix(y_test, y_pred)) # Karışıklık matrisini yazdırma

[[  4  24   1   1   0   0   0]
 [  4 454 127   8   1   0   0]
 [  1 143 313  53  10   1   0]
 [  0  14 116  80  20   4   0]
 [  0   4  41  39  17   3   0]
 [  0   2  10  15  11   7   1]
 [  0   0   1   0   1   0   0]]
